In [56]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import time
import geopandas as gdp
from cartoframes.viz import Map, Layer, popup_element
import cartoframes
from dotenv import load_dotenv
from getpass import getpass
import re
import keplergl
import plotly.express as px
import folium
import json
import requests
from folium import Choropleth, Circle, Marker, Icon, Map, TileLayer
from folium.plugins import HeatMap, MarkerCluster


In [2]:
client = MongoClient("localhost:27017")

db = client["ironhack"]


In [3]:
db.list_collection_names()

['nyrest', 'Crunchbase', 'restaurants', 'nyneigh']

In [4]:
c = db.get_collection("Crunchbase")

In [5]:
#company_category = set()

In [6]:
#df_tech_general = pd.DataFrame()

In [7]:
#def tech_df_function():

tech_name_regex = {"name":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}
tech_category_regex ={"category_code":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}
tech_description_regex = {"description":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}
tech_tags_regex = {"tag_list":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}

tech_cat_or_descr_or_tags = {"$or": [tech_name_regex, tech_category_regex, tech_description_regex, tech_tags_regex]}

projection_name_category_tags = {"_id" : 0, 
                                    "name" : 1, 
                                    "category_code": 1, 
                                    "tag_list": 1,
                                    "total_money_raised": 1,
                                    "offices.country_code" : 1, 
                                    "offices.city" : 1, 
                                    "offices.zip_code" : 1, 
                                    "offices.latitude" : 1, 
                                    "offices.longitude" : 1}

tech_companies_list = list(c.find(tech_cat_or_descr_or_tags, projection_name_category_tags).sort("offices.city", 1))
df_tech_general = pd.DataFrame(tech_companies_list)

    #return df_tech_general

In [8]:
#tech_df_function()

In [9]:
df_tech_general.shape

(285, 5)

In [10]:
def clean_monetary_values(dataframe, column):
    
    for index, row in dataframe.iterrows():
        if pd.isnull(row[column]):
            continue
        
        match = re.search('(\d+\.\d+|\d+)([MmKk])', row[column])
        if not match:
            dataframe.at[index, column] = pd.np.nan
            continue
        
        value, letter = match.groups()
        value = float(value)
        if value <= 0:
            dataframe.at[index, column] = pd.np.nan
            continue
        
        if letter.upper() == 'M':
            value *= 1000000
        else:
            value *= 1000
        
        dataframe.at[index, column] = value
    
    return dataframe


In [11]:
clean_monetary_values(df_tech_general, "total_money_raised")

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\4024254625.py:9: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  dataframe.at[index, column] = pd.np.nan


,name,category_code,tag_list,total_money_raised,offices
0,Sparter,games_video,"gaming, game, wow, worldofwarcraft, virtualgoods",NaN,"[{'zip_code': None, 'city': None, 'country_cod..."
1,Devunity,web,"techcrunch50, tc50",100000.0,[]
2,TechJuicer,None,None,NaN,[]
3,Lockergnome,None,"tech-news, it-news, blog",NaN,[]
4,Apperceptive,web,design,NaN,[]
...,...,...,...,...,...
280,Pacecode Technologies,consulting,"website-development-in-india, website-design-i...",NaN,"[{'zip_code': '600020', 'city': 'chennai', 'co..."
281,MakeMyLink,advertising,"technology, web-design, seo, software-blog, guide",NaN,"[{'zip_code': '110075', 'city': 'new delhi', '..."
282,Tech Support 4 NYC,consulting,"computer-consulting, tech-support, new-york-city",NaN,"[{'zip_code': '10018', 'city': 'new york', 'co..."
283,GoPlanit,web,"techcrunch50, tc50, travel, trip-planner, mobi...",500000.0,"[{'zip_code': '94111', 'city': 'san francisco'..."


In [12]:
#def design_df_function():
   
design_name_regex = {"name":{"$regex":"design", "$options":"i"}}
design_category_regex ={"category_code":{"$regex":"design", "$options":"i"}}
design_description_regex = {"description":{"$regex":"design", "$options":"i"}}
design_tags_regex = {"tag_list":{"$regex":"design", "$options":"i"}}

design_cat_or_descr_or_tags = {"$or": [design_name_regex, design_category_regex, design_description_regex, design_tags_regex]}

design_projection_name_category_tags = {"_id" : 0, 
                                            "name" : 1, 
                                            "category_code": 1, 
                                            "tag_list": 1,
                                            "offices.country_code" : 1, 
                                            "offices.city" : 1, 
                                            "offices.zip_code" : 1, 
                                            "offices.latitude" : 1, 
                                            "offices.longitude" : 1}

design_companies_list = list(c.find(design_cat_or_descr_or_tags, design_projection_name_category_tags).sort("offices.city", 1))
df_design = pd.DataFrame(design_companies_list)
#    return df_design

In [54]:
#design_df_function()

In [13]:
def split_dic_df_columns(dataframe, dataframe_column):
    
    """This funcion takes 2 parameters:
    1. name of given dataframe
    2. name of column that contains dictionaries as values

    It will iterate over the rows, then the specified column, checking for missing values.
    If the values is not missing, then iterate over each 
    dictionary item, and chek if the column to be created already exists.
    Then initialize the column values to zero, and then update values
    with the ones corresponding in the dictionary.
    return the original dataframe updated with a new column per each 
    key in the dictionaries, and its values.
    """
    for i, row in dataframe.iterrows():
        for j in row[dataframe_column]:
            if not pd.isnull(j):
                for x, y in j.items():
                    if x not in dataframe.columns:
                        dataframe[str(x)] = np.nan
                    dataframe.at[i, x] = y
    return dataframe



In [14]:
split_dic_df_columns(df_design, "offices")

,name,category_code,tag_list,offices,zip_code,city,country_code,latitude,longitude
0,Jasper Design,None,None,[],NaN,NaN,NaN,NaN,NaN
1,Trunkt,ecommerce,"art, design, directory, marketplace, wholesale",[],NaN,NaN,NaN,NaN,NaN
2,Apperceptive,web,design,[],NaN,NaN,NaN,NaN,NaN
3,Medium Design Group,None,None,[],NaN,NaN,NaN,NaN,NaN
4,BootB,advertising,"online-marketplace, creativity, marketing-serv...",[],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
774,ecreative,other,"design, freelance-","[{'zip_code': '80300', 'city': 'istanbul', 'co...",80300,istanbul,TUR,NaN,NaN
775,magento xperts,ecommerce,"magento-company, magento-themes, magento-templ...","[{'zip_code': '700064', 'city': 'kolkata', 'co...",700064,kolkata,IND,NaN,NaN
776,MakeMyLink,advertising,"technology, web-design, seo, software-blog, guide","[{'zip_code': '110075', 'city': 'new delhi', '...",110075,new delhi,IND,NaN,NaN
777,25 Pixels Media,other,"web-design, costa-rica, blogs, network, spanis...","[{'zip_code': '', 'city': 'san jose', 'country...",,san jose,CRI,NaN,NaN


In [15]:
split_dic_df_columns(df_tech_general, "offices")

,name,category_code,tag_list,total_money_raised,offices,zip_code,city,country_code,latitude,longitude
0,Sparter,games_video,"gaming, game, wow, worldofwarcraft, virtualgoods",NaN,"[{'zip_code': None, 'city': None, 'country_cod...",NaN,NaN,USA,37.090240,-95.712891
1,Devunity,web,"techcrunch50, tc50",100000.0,[],NaN,NaN,NaN,NaN,NaN
2,TechJuicer,None,None,NaN,[],NaN,NaN,NaN,NaN,NaN
3,Lockergnome,None,"tech-news, it-news, blog",NaN,[],NaN,NaN,NaN,NaN,NaN
4,Apperceptive,web,design,NaN,[],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
280,Pacecode Technologies,consulting,"website-development-in-india, website-design-i...",NaN,"[{'zip_code': '600020', 'city': 'chennai', 'co...",600020,chennai,IND,NaN,NaN
281,MakeMyLink,advertising,"technology, web-design, seo, software-blog, guide",NaN,"[{'zip_code': '110075', 'city': 'new delhi', '...",110075,new delhi,IND,NaN,NaN
282,Tech Support 4 NYC,consulting,"computer-consulting, tech-support, new-york-city",NaN,"[{'zip_code': '10018', 'city': 'new york', 'co...",10018,new york,USA,40.754142,-73.988360
283,GoPlanit,web,"techcrunch50, tc50, travel, trip-planner, mobi...",500000.0,"[{'zip_code': '94111', 'city': 'san francisco'...",94111,san francisco,USA,37.791576,-122.399363


In [16]:
df_design.sort_values(["city", "zip_code"], axis=0, ascending=True, na_position="first")

,name,category_code,tag_list,offices,zip_code,city,country_code,latitude,longitude
0,Jasper Design,None,None,[],NaN,NaN,NaN,NaN,NaN
1,Trunkt,ecommerce,"art, design, directory, marketplace, wholesale",[],NaN,NaN,NaN,NaN,NaN
2,Apperceptive,web,design,[],NaN,NaN,NaN,NaN,NaN
3,Medium Design Group,None,None,[],NaN,NaN,NaN,NaN,NaN
4,BootB,advertising,"online-marketplace, creativity, marketing-serv...",[],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
775,magento xperts,ecommerce,"magento-company, magento-themes, magento-templ...","[{'zip_code': '700064', 'city': 'kolkata', 'co...",700064,kolkata,IND,NaN,NaN
776,MakeMyLink,advertising,"technology, web-design, seo, software-blog, guide","[{'zip_code': '110075', 'city': 'new delhi', '...",110075,new delhi,IND,NaN,NaN
777,25 Pixels Media,other,"web-design, costa-rica, blogs, network, spanis...","[{'zip_code': '', 'city': 'san jose', 'country...",,san jose,CRI,NaN,NaN
704,A Beautiful Site,enterprise,"cms, website, websites, web-design, web-develo...","[{'zip_code': '1332', 'city': 'Sidlamafa', 'co...",,seattle,USA,47.620973,-122.347276


In [17]:
df_design["city"].value_counts().head(15)

London           21
                 19
New York         17
San Francisco    15
Chicago          13
San Diego        10
Los Angeles       8
Chennai           8
Bangalore         8
San Jose          7
Atlanta           6
Miami             6
Toronto           5
Kolkata           5
Palo Alto         5
Name: city, dtype: int64

In [18]:
df_tech_general_1M = df_tech_general[df_tech_general["total_money_raised"] >= 1000000.0]

In [19]:
df_design.dropna(subset=["city", "latitude", "longitude"], inplace=True)

In [20]:
df_tech_general_1M.dropna(subset=["city", "latitude", "longitude"], inplace=True )

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\3641945092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tech_general_1M.dropna(subset=["city", "latitude", "longitude"], inplace=True )


In [ ]:
df_tech_general_1M.sort_values(["city", "zip_code"], axis=0, ascending=True, na_position="first")

In [23]:
df_design.shape

(438, 9)

In [24]:
df_tech_general_1M.shape

(45, 10)

In [25]:
df_design = df_design[df_design["city"] != ""]

In [26]:
df_design.shape

(430, 9)

In [27]:
df_tech_general_1M["city"].value_counts().head(15)

San Francisco    12
San Mateo         3
London            2
Santa Barbara     2
Sunnyvale         2
Boston            2
Seattle           2
Gothenburg        2
Palo Alto         2
Austin            2
Montreal          1
Santa Monica      1
Reston            1
New York          1
                  1
Name: city, dtype: int64

In [28]:
df_design["city"].value_counts().head(15)

New York         15
San Francisco    10
London           10
Chicago          10
San Diego         8
Los Angeles       7
San Jose          6
Miami             6
Houston           4
Melbourne         4
Palo Alto         4
Seattle           4
Rochester         4
Bangalore         4
Birmingham        3
Name: city, dtype: int64

In [29]:


# filter the data to only include companies in San Francisco
design_sf = df_design[df_design['city'] == 'San Francisco']
tech_sf = df_tech_general_1M[df_tech_general_1M['city'] == 'San Francisco']

# combine the data into one dataframe
df_sf = pd.concat([design_sf, tech_sf], ignore_index=True)



In [30]:
df_sf.shape

(22, 10)

In [31]:
df_sf.head(22)

,name,category_code,tag_list,offices,zip_code,city,country_code,latitude,longitude,total_money_raised
0,Popego,advertising,"techcrunch50, tc50, web-service, interests, se...","[{'zip_code': '1414BHH', 'city': 'Buenos Aires...",94102,San Francisco,USA,37.778687,-122.421242,NaN
1,Popego,advertising,"techcrunch50, tc50, web-service, interests, se...","[{'zip_code': '1414BHH', 'city': 'Buenos Aires...",94102,San Francisco,USA,37.778687,-122.421242,NaN
2,Netbiscuits,mobile,"mobile-web, cloud-platform, adaptive-web-techn...","[{'zip_code': '67657', 'city': 'Kaiserslautern...",94105,San Francisco,USA,37.755037,-122.426442,NaN
3,Digg,news,"community, social, news, bookmark, digg, techn...","[{'zip_code': '94107', 'city': 'San Francisco'...",94107,San Francisco,USA,37.764726,-122.394523,NaN
4,Cutcaster,web,"vectors, image, stock-photography, photo, phot...","[{'zip_code': '94129', 'city': 'San Francisco'...",94129,San Francisco,USA,37.798510,-122.457454,NaN
5,Minted,ecommerce,"crowd-sourcing, design, art, stationery, ecomm...","[{'zip_code': '94111', 'city': 'San Francisco'...",94111,San Francisco,USA,37.797435,-122.403175,NaN
6,Milobox,advertising,"real-estate, web-2-0, realtor-web-sites, real-...","[{'zip_code': '94109', 'city': 'San Francisco'...",94109,San Francisco,USA,37.792979,-122.421242,NaN
7,Milobox,advertising,"real-estate, web-2-0, realtor-web-sites, real-...","[{'zip_code': '94109', 'city': 'San Francisco'...",94109,San Francisco,USA,37.792979,-122.421242,NaN
8,5th Finger,mobile,,"[{'zip_code': '94108', 'city': 'San Francisco'...",94108,San Francisco,USA,37.790099,-122.402199,NaN
9,Bamboudesign,mobile,"mobile-apps, ux-design, user-experience-design...","[{'zip_code': '94103', 'city': 'San Francisco'...",94103,San Francisco,USA,37.775087,-122.415199,NaN


In [33]:
# create the Kepler.gl map
map_1 = keplergl.KeplerGl(height=600, data={'sf_companies': df_sf})
map_1


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\germa\miniconda3\envs\ironhack\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'sf_companies':                              name category_code  \
0                          P…

In [34]:
# filter the data to only include companies in London
design_london = df_design[df_design['city'] == 'London']
tech_london = df_tech_general_1M[df_tech_general_1M['city'] == 'London']

# combine the data into one dataframe
df_london = pd.concat([design_london, tech_london], ignore_index=True)

# create the Kepler.gl map
map_2 = keplergl.KeplerGl(height=600, data={'london_companies': df_london})
map_2


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\germa\miniconda3\envs\ironhack\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'london_companies':                         name     category_code  \
0                     cSc…

In [35]:
df_london.shape

(12, 10)

In [36]:
df_london.head(25)

,name,category_code,tag_list,offices,zip_code,city,country_code,latitude,longitude,total_money_raised
0,cScape,web,"customer-engagement, sharepoint, microsoft-par...","[{'zip_code': 'EC1R 0DS', 'city': 'London', 'c...",EC1R 0DS,London,GBR,51.523676,-0.107843,NaN
1,ProofHQ,software,"proofhq, online-proofing, review, feedback, co...","[{'zip_code': '', 'city': 'London', 'country_c...",,London,GBR,51.532048,-0.011154,NaN
2,Sososher,web,"sososher, shop, shopping, design, webstart, fa...","[{'zip_code': '', 'city': 'London', 'country_c...",,London,GBR,51.500152,-0.126236,NaN
3,The Tin,web,"digital-consultancy, solutions, technology, de...","[{'zip_code': 'EC1N 8JH', 'city': 'London', 'c...",EC1N 8JH,London,GBR,51.521290,-0.109780,NaN
4,Which Web Design Company,enterprise,"website-design, review, web-design, rating, we...","[{'zip_code': '', 'city': 'London', 'country_c...",,London,GBR,51.500152,-0.126236,NaN
5,Make It Rain,advertising,"ppc, seo, affiliates, web-design, social-media...","[{'zip_code': 'SW4 0BG', 'city': 'London', 'co...",SW4 0BG,London,GBR,51.462600,-0.139082,NaN
6,Masterpiece,advertising,"creative, online, web, cms, content-management...","[{'zip_code': 'EC2A 4LD', 'city': 'London', 'c...",EC2A 4LD,London,GBR,51.521913,-0.083375,NaN
7,Boring,other,"web-design, social-media, production, consulta...","[{'zip_code': 'E1 6PX', 'city': 'London', 'cou...",E1 6PX,London,GBR,51.518372,-0.074068,NaN
8,Marmalade on Toast,public_relations,,"[{'zip_code': '', 'city': 'London', 'country_c...",,London,GBR,51.500152,-0.126236,NaN
9,Red Engineering Design,consulting,"mep, mechanical-engineering, electrical-engine...","[{'zip_code': 'W1T 3NJ', 'city': 'London', 'co...",W1T 3NJ,London,GBR,51.523421,-0.139861,NaN


In [49]:


# create the Folium map
folium_sf_map = folium.Map(location=[37.7749, -122.4194], zoom_start=10)

# add markers for each company in San Francisco
for i, row in df_sf.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name']
    ).add_to(folium_sf_map)

# display the map
folium_sf_map


In [50]:
df_sf.sample()

,name,category_code,tag_list,offices,zip_code,city,country_code,latitude,longitude,total_money_raised
16,AdRocket,advertising,"techcrunch50, tc50, email, ad-network, targeti...","[{'zip_code': '', 'city': 'San Francisco', 'co...",,San Francisco,USA,37.775196,-122.419204,1000000.0


In [63]:
df_sf["latitude"].dtypes

dtype('float64')

In [52]:
token = getpass()

In [53]:
import requests

url = "https://api.foursquare.com/v3/places/search?ll=37.775196%2C-122.419204&radius=600&categories=12055%2C%2012056%2C%2012057%2C%2012058%2C%2012059%2C%2012060"

headers = {
    "accept": "application/json",
    "Authorization": token
}

response = json.requests.get(url, headers=headers)

print(response)

{"results":[{"fsq_id":"540270a7498ee01256291abf","categories":[{"id":12056,"name":"Preschool","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/school_","suffix":".png"}}],"chains":[],"distance":146,"geocodes":{"main":{"latitude":37.776344,"longitude":-122.418868},"roof":{"latitude":37.776344,"longitude":-122.418868}},"link":"/v3/places/540270a7498ee01256291abf","location":{"address":"50 Fell St","census_block":"060750124052005","country":"US","cross_street":"","dma":"San Francisco-Oakland-San Jose","formatted_address":"50 Fell St, San Francisco, CA 94102","locality":"San Francisco","postcode":"94102","region":"CA"},"name":"LePort School","related_places":{},"timezone":"America/Los_Angeles"},{"fsq_id":"5da0abe01ebd700008209749","categories":[{"id":12056,"name":"Preschool","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/school_","suffix":".png"}}],"chains":[],"distance":159,"geocodes":{"main":{"latitude":37.776653,"longitude":-122.418671},"roof":{"latit

In [69]:
I need code in python. After reading this, please tell me the code if you think it's a good idea to make it all into a function.
I have a dataframe called df_sf where I have 10 columns listing companies in san francisco. From columns "latitude" and "longitude", 
I need to get the float values to pass them for an API call.
Here you can see an example of the api call I will be performing.

url = "https://api.foursquare.com/v3/places/search?ll=37.775196%2C-122.419204&radius=600&categories=12055%2C%2012056%2C%2012057%2C%2012058%2C%2012059%2C%2012060"
headers = {"accept": "application/json", "Authorization": token}
response = json.requests.get(url, headers=headers)

This API call, gives me as a response, a json where I can find the info I need about ten venues corresponding to the specified categories 
within a give radius starting from the location of each company. This location is declared by the latitude and longitude I mentioned before.
I will use the exact same call, but I need to iterate the values for latitude and longitude, which you can see in the given example
as 37.775196 for latitude and -122.419204 for longitude.

I used to have this function for getting from the response of the API only one name and its latitude and longitude. I hope it serves you as a reference for what I need:

def name_coordinates (dict_):
        processed_dict = {"name": dict_["name"],
                     "latitude": dict_["geocodes"]["main"]["latitude"],
                     "longitude": dict_["geocodes"]["main"]["longitude"]}
        return processed_dict
I implemented it like this:
name_coordinates (response.json()["results"][0])

I want a new dataframe called df_sf_schools with the following outcome, which I really don't know how to organize:

for each company in the df_sf, its name (column 'name'), the names of nearby schools (one per row, of course) that we get from the API call, 
and latitude and longitude for each school. 
For this task, is it a good idea to do some grouping or pivoting or something like this?
Thanks

SyntaxError: unterminated string literal (detected at line 1) (93124809.py, line 1)

In [59]:
len(response.json()["results"])

10

In [60]:
def name_coordinates (dict_):
    
    processed_dict = {"name": dict_["name"],
                     "latitude": dict_["geocodes"]["main"]["latitude"],
                     "longitude": dict_["geocodes"]["main"]["longitude"]}
    
    return processed_dict

In [61]:
name_coordinates (response.json()["results"][0])

{'name': 'LePort School', 'lat': 37.776344, 'lon': -122.418868}

In [ ]:
df_sf_venues = 

In [ ]:
def update_df_with_api(df, json_dic, )

In [ ]:


#def get_nearby_schools(df_sf, token, categories, radius):
#    schools = []
#    for index, row in df_sf.iterrows():
#        lat = row["latitude"]
#        lng = row["longitude"]
#        url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{lng}&radius={radius}&categories={categories}"
#        headers = {"accept": "application/json", "Authorization": token}
#        response = requests.get(url, headers=headers)
#        data = response.json()
#        for result in data["results"]:
#            schools.append({"name": row["name"], "school_name": result["name"], "latitude": result["location"]["lat"], "longitude": result["location"]["lng"]})
#    return pd.DataFrame(schools)


In [66]:


def get_nearby_schools(df_sf):
    df_sf_schools = pd.DataFrame(columns=['name', 'school_names', 'school_latitudes', 'school_longitudes'])
    
    for i, row in df_sf.iterrows():
        latitude = row['latitude']
        longitude = row['longitude']
        url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&radius=600&categories=12055%2C%2012056%2C%2012057%2C%2012058%2C%2012059%2C%2012060"
        headers = {"accept": "application/json", "Authorization": token}
        response = requests.get(url, headers=headers)
        results = response.json()["results"]
        
        school_names = []
        school_latitudes = []
        school_longitudes = []
        for result in results:
            school_names.append(result["name"])
            school_latitudes.append(result["geocodes"]["main"]["latitude"])
            school_longitudes.append(result["geocodes"]["main"]["longitude"])
        
        df_sf_schools = df_sf_schools.append({'name': row['name'],
                                              'school_names': school_names,
                                              'school_latitudes': school_latitudes,
                                              'school_longitudes': school_longitudes
                                             }, ignore_index=True)
    return df_sf_schools


In [67]:
get_nearby_schools(df_sf)

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\3732514187.py:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\3732514187.py:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\3732514187.py:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\3732514187.py:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\3732514187.py:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

,name,school_names,school_latitudes,school_longitudes
0,Popego,"[San Francisco Unified School District, Civic ...","[37.779265, 37.780381, 37.780419, 37.780887, 3...","[-122.422123, -122.422906, -122.422836, -122.4..."
1,Popego,"[San Francisco Unified School District, Civic ...","[37.779265, 37.780381, 37.780419, 37.780887, 3...","[-122.422123, -122.422906, -122.422836, -122.4..."
2,Netbiscuits,"[City College of San Francisco Mission Campus,...","[37.755083, 37.754782, 37.75515, 37.755248, 37...","[-122.420049, -122.426122, -122.42815, -122.42..."
3,Digg,"[Daniel Webster Elementary SCHL, Live Oak Scho...","[37.760539, 37.763563, 37.763525, 37.760604, 3...","[-122.396075, -122.399263, -122.399333, -122.3..."
4,Cutcaster,"[Cow Hollow School, German International Schoo...","[37.799073, 37.800017, 37.800623, 37.797795, 3...","[-122.455624, -122.45547, -122.455859, -122.46..."
5,Minted,"[John Yehall Chin Elementary School, Saint Mar...","[37.798486, 37.795777, 37.79478, 37.794704, 37...","[-122.403206, -122.404818, -122.404185, -122.4..."
6,Milobox,"[St Luke's School, Spring Valley Elementary Sc...","[37.792097, 37.794215, 37.795198, 37.789619, 3...","[-122.422114, -122.418849, -122.424786, -122.4..."
7,Milobox,"[St Luke's School, Spring Valley Elementary Sc...","[37.792097, 37.794215, 37.795198, 37.789619, 3...","[-122.422114, -122.418849, -122.424786, -122.4..."
8,5th Finger,"[Decker Communications, Notre Dame Des Victoir...","[37.789616, 37.791201, 37.793289, 37.79478, 37...","[-122.400368, -122.406555, -122.405316, -122.4..."
9,Bamboudesign,"[Presidio Knolls School, Judith Baker Child De...","[37.773306, 37.776862, 37.776344, 37.776653, 3...","[-122.413743, -122.411659, -122.418868, -122.4..."


In [71]:


def get_nearby_schools(df_sf):
    df_sf_schools = pd.DataFrame(columns=["Company Name", "School Name", "Latitude", "Longitude"])
    
    for i, row in df_sf.iterrows():
        latitude = row["latitude"]
        longitude = row["longitude"]
        
        url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&radius=600&categories=12055%2C%2012056%2C%2012057%2C%2012058%2C%2012059%2C%2012060"
        headers = {"accept": "application/json", "Authorization": token}
        response = requests.get(url, headers=headers)
        
        for school in response.json()["results"]:
            df_sf_schools = df_sf_schools.append({"Company Name": row["name"], 
                                                  "School Name": school["name"], 
                                                  "Latitude": school["geocodes"]["main"]["latitude"], 
                                                  "Longitude": school["geocodes"]["main"]["longitude"]}, 
                                                 ignore_index=True)
            
    return df_sf_schools


In [72]:
df_sf_schools = get_nearby_schools(df_sf)

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\2518860685.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\2518860685.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\2518860685.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\2518860685.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\germa\AppData\Local\Temp\ipykernel_10908\2518860685.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [73]:
df_sf_schools

,Company Name,School Name,Latitude,Longitude
0,Popego,San Francisco Unified School District,37.779265,-122.422123
1,Popego,Civic Center Secondary School,37.780381,-122.422906
2,Popego,John Swett Alternative Elementary School,37.780419,-122.422836
3,Popego,C5 Children's School,37.780887,-122.419280
4,Popego,Mission Montessori,37.776653,-122.418671
...,...,...,...,...
179,Canopy Financial,S3 Social Media Academy,37.791189,-122.395155
180,Canopy Financial,Holistic Seedlings Wonderschool,37.788832,-122.393941
181,Canopy Financial,Marin Day Schools,37.789407,-122.388956
182,Canopy Financial,Oxford Elementary School,37.791241,-122.396416


In [74]:
df_sf_schools.to_csv("San_Francisco_Schools.csv", index=False)